In [1]:
# Import libaries
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Step 1: Create a soup object from the home page

In [2]:
url = 'http://chubbygrub.com/'
res = requests.get(url)
print(res.status_code)

soup = BeautifulSoup(res.content, 'lxml')

200


### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and slug. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'slug': 'aw-restaurants'}, 
    {'name': "Applebee's", 'slug': 'applebees'},
    ...
]
```

In [3]:
restaurants = []

for li in soup.findAll('a',{'class', 'btn'}):
    restaurant = {}
    restaurant['name'] = li.text
    restaurant['slug'] = li['href'][13:]
    restaurants.append(restaurant)

restaurants

[{'name': 'A&W Restaurants', 'slug': 'aw-restaurants'},
 {'name': "Applebee's", 'slug': 'applebees'},
 {'name': "Arby's", 'slug': 'arbys'},
 {'name': 'Atlanta Bread Company', 'slug': 'atlanta-bread-company'},
 {'name': "Bojangle's Famous Chicken 'n Biscuits",
  'slug': 'bojangles-famous-chicken-n-biscuits'},
 {'name': 'Buffalo Wild Wings', 'slug': 'buffalo-wild-wings'},
 {'name': 'Burger King', 'slug': 'burger-king'},
 {'name': "Captain D's", 'slug': 'captain-ds'},
 {'name': "Carl's Jr.", 'slug': 'carls-jr'},
 {'name': "Charley's Grilled Subs", 'slug': 'charleys-grilled-subs'},
 {'name': 'Chick-fil-A', 'slug': 'chick-fil-a'},
 {'name': "Chili's", 'slug': 'chilis'},
 {'name': 'Chipotle Mexican Grill', 'slug': 'chipotle-mexican-grill'},
 {'name': "Church's", 'slug': 'churchs'},
 {'name': 'Corner Bakery Cafe', 'slug': 'corner-bakery-cafe'},
 {'name': 'Dairy Queen', 'slug': 'dairy-queen'},
 {'name': "Denny's", 'slug': 'dennys'},
 {'name': 'El Pollo Loco', 'slug': 'el-pollo-loco'},
 {'name'

### Step 3: Using the slug, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [4]:
%%time
foods = []

for entry in restaurants:
    url = 'http://chubbygrub.com/restaurants/' + entry['slug']
    res = requests.get(url)
    if res.status_code != 200:
        print('request error')
        break
    soup = BeautifulSoup(res.content, 'lxml')
    for block in soup.find('tbody').find_all('tr'):
        food_item = {}
        food_item['Name'] = block.find_all('td')[0].text.strip()
        food_item['Category'] = block.find_all('td')[1].text.strip()
        food_item['Calories'] = block.find_all('td')[2].text.strip()
        food_item['Fat'] = block.find_all('td')[3].text.strip()
        food_item['Carbohydrates'] = block.find_all('td')[4].text.strip()
        food_item['Restaurant'] = entry['name']
        foods.append(food_item)

CPU times: user 6.61 s, sys: 208 ms, total: 6.81 s
Wall time: 1min 30s


In [5]:
foods[:3]

[{'Name': 'A&W® Diet Root Beer',
  'Category': 'Drinks',
  'Calories': '0',
  'Fat': '0',
  'Carbohydrates': '0',
  'Restaurant': 'A&W Restaurants'},
 {'Name': 'A&W® Diet Root Beer',
  'Category': 'Drinks',
  'Calories': '0',
  'Fat': '0',
  'Carbohydrates': '0',
  'Restaurant': 'A&W Restaurants'},
 {'Name': 'A&W® Diet Root Beer',
  'Category': 'Drinks',
  'Calories': '0',
  'Fat': '0',
  'Carbohydrates': '0',
  'Restaurant': 'A&W Restaurants'}]

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 4,977 rows

In [7]:
df = pd.DataFrame(foods)
df.head()

,Calories,Carbohydrates,Category,Fat,Name,Restaurant
0,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
1,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
2,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
3,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
4,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [8]:
df.to_csv('FoodScrape.csv', index=False)